# Read in SAM data and save as netcdfs 

In [1]:
import xarray as xr, matplotlib.pyplot as plt
from importlib import reload # need to use this if I edit a function file
import os
import numpy as np
import pandas as pd
import cartopy.crs as ccrs # to add in continents and change map projections 
from matplotlib.colors import LinearSegmentedColormap # to change colour bar????
import dask.diagnostics # dask allows you to check how long something is taking to load
import climtas # needed to count event statistics with a specified duration
import csv

In [2]:
# import custom functions
import sys 
sys.path.append('/home/563/kb6999/Functions') # use this if the function file is in a different directory to the notebook

import frequently_used_functions as func
import plotting_functions as fplot
import model_functions as funcM
import reanalysis_functions as funcR

## read in reanalysis pressure data to caluclate SAM

In [7]:
# !ls /g/data/ua8/LE_models/20CRv3/mean_daily/prmsl

In [6]:
path_p = '/g/data/ua8/LE_models/20CRv3/mean_daily/prmsl/*.nc'

In [8]:
# read in daily mean sea level pressure (mean across all 20CR members)
P_ds = xr.open_mfdataset(path_p)
del P_ds['time_bnds']
P_ds

<xarray.Dataset>
Dimensions:  (lat: 181, lon: 360, time: 65744)
Coordinates:
  * lat      (lat) float32 -90.0 -89.0 -88.0 -87.0 -86.0 ... 87.0 88.0 89.0 90.0
  * lon      (lon) float32 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
  * time     (time) datetime64[ns] 1836-01-01 1836-01-02 ... 2015-12-31
Data variables:
    prmsl    (time, lat, lon) float32 dask.array<chunksize=(366, 181, 360), meta=np.ndarray>
Attributes: (12/24)
    Conventions:               CF-1.2
    title:                     mean Daily NOAA/CIRES/DOE 20th Century Reanaly...
    comments:                  Data are from \nNOAA/CIRES/DOE 20th Century Re...
    standard_name_vocabulary:  NetCDF Climate and Forecast (CF) Metadata Conv...
    license:                   These data are available free of charge under ...
    spatial_resolution:        1.0 degree
    ...                        ...
    citation1:                 Slivinski, L. C, G. P. Compo, J. S. Whitaker, ...
    References:                https://www.psl.noaa.gov/data/gridded/data.20t...
    creator_name:              NOAA/PSL
    institution:               NOAA Physical Sciences Laboratory & CU/CIRES \...
    contact:                   psl.data@noaa.gov
    citation:                  Compo,G.P. <https://www.psl.noaa.gov/people/gi...

In [9]:
# resample to monthly
P_mon = P_ds.resample(time='M').min(dim='time')
P_mon

<xarray.Dataset>
Dimensions:  (lat: 181, lon: 360, time: 2160)
Coordinates:
  * time     (time) datetime64[ns] 1836-01-31 1836-02-29 ... 2015-12-31
  * lat      (lat) float32 -90.0 -89.0 -88.0 -87.0 -86.0 ... 87.0 88.0 89.0 90.0
  * lon      (lon) float32 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
Data variables:
    prmsl    (time, lat, lon) float32 dask.array<chunksize=(1, 181, 360), meta=np.ndarray>

In [10]:
with dask.diagnostics.ProgressBar():
    P_mon.to_netcdf('MSLP_mon.nc')

[########################################] | 100% Completed |  1min 53.7s


P*40�S and P*65�S are the normalized monthly zonal sea level pressure (SLP) at 40�S and 65�S
The 30-year period from 1971-2000 is used for the normalization period. Note that this period is arbitrary and a switch to a different period would change the exact SAM values but would be unlikely to significantly alter any trends.
(true zonal mean was simply the mean of the 144 points along the appropriate parallel)

In [11]:
# read in netcdf of monthly pressure
P_mon_ds = xr.open_dataset('MSLP_mon.nc')

In [12]:
# calculate the zonal mean mslp at 40 and 65S (to be used in calculating SAM)
P40S = P_mon_ds.sel(lat=-40).mean(dim='lon')
P65S = P_mon_ds.sel(lat=-65).mean(dim='lon')
P65S

<xarray.Dataset>
Dimensions:  (time: 2160)
Coordinates:
  * time     (time) datetime64[ns] 1836-01-31 1836-02-29 ... 2015-12-31
    lat      float32 -65.0
Data variables:
    prmsl    (time) float32 9.88e+04 9.872e+04 9.846e+04 ... 9.592e+04 9.68e+04

In [13]:
# set the dates for the normalisation period
start_date = '1971'
end_date = '2000'

In [14]:
# calculate the anomaly for the normalised zonal mean, x, (x-mean)/std
anom40 = func.monthly_anomaly(P40S, start_date, end_date)
anom65 = func.monthly_anomaly(P65S, start_date, end_date)
anom65

<xarray.Dataset>
Dimensions:  (time: 2160)
Coordinates:
  * time     (time) datetime64[ns] 1836-01-31 1836-02-29 ... 2015-12-31
    lat      float32 -65.0
    month    (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    prmsl    (time) float32 1.437e+03 1.797e+03 1.856e+03 ... -868.9 -471.2

In [15]:
# calculate the std for the normalised zonal mean, x, (x-mean)/std
std40 = anom40.sel(time=slice(start_date, end_date)).std(dim='time')
std65 = anom65.sel(time=slice(start_date, end_date)).std(dim='time')
std40

<xarray.Dataset>
Dimensions:  ()
Coordinates:
    lat      float32 -40.0
Data variables:
    prmsl    float64 178.9

In [16]:
# normalise the pressure values by minusing the mean (ie calculating anomaly) and dividing by 30-year average
P40S_n = anom40/std40
P65S_n = anom65/std65

In [17]:
# calculate the SAM index by finding the normalised pressure difference between the two latitude bands
SAM = P40S_n - P65S_n
SAM

<xarray.Dataset>
Dimensions:  (time: 2160)
Coordinates:
  * time     (time) datetime64[ns] 1836-01-31 1836-02-29 ... 2015-12-31
    month    (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    prmsl    (time) float64 1.701 0.5813 1.714 2.078 ... -0.6553 1.869 0.5343

In [21]:
with dask.diagnostics.ProgressBar():
    SAM.prmsl.to_netcdf('R_SAM_index.nc')

## read in Abram and Marshall SAM index data

In [3]:
path = '/g/data/w48/kb6999'

In [19]:
# !ls /g/data/w48/kb6999

In [5]:
# read in daily mean sea level pressure (mean across all 20CR members)
P_df = pd.read_csv(f'{path}/abram2014sam_data.txt', delimiter='\t')
# set the date as teh index column
P_df = P_df.set_index(['age_AD'])
P_df

,SAM_Index,SAM_Index_err,RE,RE_crit,SAM_Index_70yloess,SAM_Index_+error_70yloess,SAM_Index_-error_70yloess,Marshall_SAM_Index
age_AD,,,,,,,,
2007,-2.655296,3.217976,-0.015027,-0.237163,1.815786,4.752763,-1.119229,-0.083
2006,2.516332,3.217976,-0.015027,-0.237163,1.762162,4.682995,-1.156849,2.197
2005,0.250082,3.236963,-0.027041,-0.115012,1.708567,4.613422,-1.194603,0.987
2004,2.043958,3.236963,-0.027041,-0.115012,1.655018,4.544066,-1.232481,1.857
2003,0.834730,3.236963,-0.027041,-0.115012,1.601538,4.474956,-1.270464,1.037
...,...,...,...,...,...,...,...,...
1004,-1.813489,3.011280,0.111179,0.006600,-1.579751,1.431294,-4.590993,NaN
1003,-1.529136,3.011280,0.111179,0.006600,-1.563468,1.447598,-4.574714,NaN
1002,-1.007336,3.011280,0.111179,0.006600,-1.548646,1.462439,-4.559895,NaN


In [6]:
# convert to xarra
P_xr = P_df.to_xarray()
P_xr

<xarray.Dataset>
Dimensions:                    (age_AD: 1008)
Coordinates:
  * age_AD                     (age_AD) int64 2007 2006 2005 ... 1002 1001 1000
Data variables:
    SAM_Index                  (age_AD) float64 -2.655 2.516 ... -1.524 -3.22
    SAM_Index_err              (age_AD) float64 3.218 3.218 ... 3.011 3.011
    RE                         (age_AD) float64 -0.01503 -0.01503 ... 0.1112
    RE_crit                    (age_AD) float64 -0.2372 -0.2372 ... 0.0066
    SAM_Index_70yloess         (age_AD) float64 1.816 1.762 ... -1.536 -1.524
    SAM_Index_+error_70yloess  (age_AD) float64 4.753 4.683 ... 1.476 1.487
    SAM_Index_-error_70yloess  (age_AD) float64 -1.119 -1.157 ... -4.547 -4.535
    Marshall_SAM_Index         (age_AD) float64 -0.083 2.197 0.987 ... nan nan

In [7]:
# rename age to time
P_time = P_xr.rename({'age_AD':'time'})
# reorder time from smallest to largest
P_time = P_time.sortby('time', ascending=True)
P_time

<xarray.Dataset>
Dimensions:                    (time: 1008)
Coordinates:
  * time                       (time) int64 1000 1001 1002 ... 2005 2006 2007
Data variables:
    SAM_Index                  (time) float64 -3.22 -1.524 ... 2.516 -2.655
    SAM_Index_err              (time) float64 3.011 3.011 3.011 ... 3.218 3.218
    RE                         (time) float64 0.1112 0.1112 ... -0.01503
    RE_crit                    (time) float64 0.0066 0.0066 ... -0.2372 -0.2372
    SAM_Index_70yloess         (time) float64 -1.524 -1.536 ... 1.762 1.816
    SAM_Index_+error_70yloess  (time) float64 1.487 1.476 1.462 ... 4.683 4.753
    SAM_Index_-error_70yloess  (time) float64 -4.535 -4.547 ... -1.157 -1.119
    Marshall_SAM_Index         (time) float64 nan nan nan ... 0.987 2.197 -0.083

In [9]:
# select out years from 1850-present
start_date, end_date = '1850','2007'
P_final = P_time.sel(time=slice(start_date, end_date))
time = pd.date_range(pd.to_datetime(start_date), pd.to_datetime(end_date), freq='YS')
P_final['time'] = time
P_final

<xarray.Dataset>
Dimensions:                    (time: 158)
Coordinates:
  * time                       (time) datetime64[ns] 1850-01-01 ... 2007-01-01
Data variables:
    SAM_Index                  (time) float64 -0.5283 -1.041 ... 2.516 -2.655
    SAM_Index_err              (time) float64 2.318 2.318 2.318 ... 3.218 3.218
    RE                         (time) float64 0.4734 0.4734 ... -0.01503
    RE_crit                    (time) float64 0.2604 0.2604 ... -0.2372 -0.2372
    SAM_Index_70yloess         (time) float64 -0.6394 -0.6412 ... 1.762 1.816
    SAM_Index_+error_70yloess  (time) float64 1.688 1.687 1.687 ... 4.683 4.753
    SAM_Index_-error_70yloess  (time) float64 -2.966 -2.968 ... -1.157 -1.119
    Marshall_SAM_Index         (time) float64 nan nan nan ... 0.987 2.197 -0.083

In [10]:
# convert to netcdf
with dask.diagnostics.ProgressBar():
    P_final.to_netcdf(f'{path}/SAM_data_abram_1850_2007.nc')